## Basic Setting

In [834]:
import pymongo
def dbconnect(collection) :
    client=pymongo.MongoClient('mongodb://trainings.iptime.org:45003/')
    db= client['data_analysis']
    collection = db[collection]
    data = collection.find()
    df = pd.DataFrame(data)
    return df


In [835]:
# example
df_dcinside = dbconnect('persona1_dcinside')

In [836]:
### 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Font directories:
	/root/.local/share/fonts
	/usr/local/share/fonts
	/usr/share/fonts
	/root/.fonts
	/usr/share/fonts/truetype
	/usr/share/fonts/truetype/dejavu
	/usr/share/fonts/truetype/liberation
	/usr/share/fonts/truetype/nanum
/root/.local/share/fonts: skipping, no such directory
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/dejavu: caching, new cache contents: 22 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/root/.fonts: skipping, no such directory
/usr/shar

In [837]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [838]:
import pandas as pd
import seaborn as sns

## 희귀질환 Data(수치형)

### Dataset
- 2021년 data
  + 희귀질환 질병분류코드별 "항목별 진료내역", "지역별 사망자수", "연령별 사망자수", "지역별 발생자수", "성별별 + 연령별 발생자수"
  + 희귀질환 건강보험 진료비 통계
- 2022년 data
  + 중증질환 "연령별 + 성별별 + 진료현황", 4대 중증질환 약품비 현황
  + 희귀질환 건강보험 진료비 통계
- 희귀질환정보 항목분류
  + 희귀질환 계열 정보 분류
  + 전처리코드
```
        def codes(x):
            condition = (df_cate["질환코드"] == x[0]) &\
                   (float(x[1:]) >= df_cate["코드시작"].astype(float)) &\
                   (float(x[1:]) <= df_cate["코드끝"].astype(float))
            result = df_cate.loc[condition, '항목분류']
            if len(result) > 0:
                return result.iloc[0]
            else:
                return None  # 여기서 None 대신 원하는 기본값을 사용하십시오.

        df_insurance['항목분류'] = df_insurance['희귀질환 상병코드'].apply(codes)
```

### 발생자 및 사망자 수 data 

#### 질병분류코드별 발생자수 data

In [839]:
df_gender_age_accrual = pd.read_csv('./csv/2021_전국_질병분류코드별_성별_연령군별_발생자_수_발생자_수_200명_초과_질환__20240424095607.csv', header=1)
df_gender_age_accrual.columns

Index(['질병코드 및 질환별(1)', '성별(1)', '발생자수', '<1', '1~9', '10~19', '20~29',
       '30~39', '40~49', '50~59', '60~69', '70~79', '80<=', '알수없음'],
      dtype='object')

##### Data 살펴보기

|value|contents|key|opinion|
|--|--|--|--|
|질병분류코드 및 질환별(1)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병|-|범주형, 코드와 질환명 분류 필요|
|성별(1)|성별 및 총계|남자, 여자, 계|남자, 여자와 계를 분리해야 함.|
|발생자수|발생자수|-|수치형|
|<1|1세 미만|-|수치형|
|1~9|1에서 9세|-|수치형|
|10~19|10에서 19세|-|수치형|
|20~29|20에서 29세|-|수치형|
|30~39|30에서 39세|-|수치형|
|40~49|40에서 49세|-|수치형|
|50~59|50에서 59세|-|수치형|
|60~69|60에서 69세|-|수치형|
|70~79|70에서 79세|-|수치형|
|80<=|80 이상|-|수치형|
|알수없음|알수없음|-|수치형이나 0밖에 없음 - 삭제 필요|

##### 기본 전처리

In [840]:
def split_codes(x):
    # 문자열을 공백을 기준으로 두 부분으로 나누기
    parts = x.split(maxsplit=1)

    if x.split(maxsplit=1)[0]  not in ('사례별', '사례별로','증후군의','진단내용에','진단된','질병의','신생물', '1형:', '질병분류코드 및 질환별(2)', '질병분류코드'):
        parts = x.split(maxsplit=1)
        return pd.Series(parts)
    # 두 부분으로 나눌 수 있으면 해당 값을 반환, 그렇지 않으면 code만 반환하고 name은 빈 문자열로 처리
    else:
        return pd.Series(['', x])  # name을 빈 문자열로 처리

In [841]:
df_gender_age_accrual[['질병코드', '질병명']] = df_gender_age_accrual['질병코드 및 질환별(1)'].apply(split_codes)

In [842]:
df_gender_age_accrual = df_gender_age_accrual.rename(columns={'질병코드 및 질환별(1)': '질병코드 및 질환', '성별(1)': '성별', '<1': '1세이하', '1~9': '1~9세', '10~19': '10~19세', '20~29': '20~29세', '30~39': '30~39세', '40~49': '40~49세', '50~59': '50~59세', '60~69': '60~69세', '70~79': '70~79세', '80<=': '80세 이상'})
df_gender_age_accrual = df_gender_age_accrual.drop('알수없음', axis=1)

In [843]:
df_gender_age_accrual.columns

Index(['질병코드 및 질환', '성별', '발생자수', '1세이하', '1~9세', '10~19세', '20~29세', '30~39세',
       '40~49세', '50~59세', '60~69세', '70~79세', '80세 이상', '질병코드', '질병명'],
      dtype='object')

In [844]:
df_gender_age_accrual = df_gender_age_accrual[['질병코드', '질병명', '성별', '발생자수', '1세이하', '1~9세', '10~19세', '20~29세', '30~39세',
       '40~49세', '50~59세', '60~69세', '70~79세', '80세 이상', ]]

In [845]:
df_gender_age_accrual[:5]

,질병코드,질병명,성별,발생자수,1세이하,1~9세,10~19세,20~29세,30~39세,40~49세,50~59세,60~69세,70~79세,80세 이상
0,계,NaN,계,43079,466,799,2272,4196,4344,5345,7072,8618,6602,3365
1,D61.3,특발성 무형성빈혈,계,406,1,14,30,46,36,51,65,74,65,24
2,D61.3,특발성 무형성빈혈,남자,186,0,8,17,22,12,26,26,37,30,8
3,D61.3,특발성 무형성빈혈,여자,220,1,6,13,24,24,25,39,37,35,16
4,D61.9,저형성빈혈 NOS,계,223,0,13,18,18,29,27,31,42,26,19


In [846]:
# CSV 파일로 내보내기
df_gender_age_accrual.to_csv('./New_csv(데이터전처리 완료된 csv)/2021_질병분류코드별_발생자수.csv', index=False, encoding='utf-8')

#### 지역별 질병분류코드별 발생자수 data

In [847]:
df_region_accrual = pd.read_csv('./csv/2021_지역별_질병분류코드별_발생자_수_발생자_수_200명_초과_질환__20240424095500.csv', header=1)

##### Data 살펴보기

|value|contents|key|opinion|
|--|--|--|--|
|질병분류코드 및 질환별(1)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병, 계|-|범주형, 코드와 질환명 분류 필요, 계 정보 따로 빼야 함|
|발생자 수|발생자 수|-|수치형|
|서울특별시|서울특별시|-|수치형|
|인천광역시|인천광역시|-|수치형|
|경기도|경기도|-|수치형|
|강원도|강원도|-|수치형|
|충청남도|충청남도|-|수치형|
|충청북도|충청북도|-|수치형|
|대전광역시|대전광역시|-|수치형|
|세종특별자치시|세종특별자치시|-|수치형|
|부산광역시|부산광역시|-|수치형|
|대구광역시|대구광역시|-|수치형|
|울산광역시|울산광역시|-|수치형|
|전라북도|전라북도|-|수치형|
|전라남도|전라남도|-|수치형|
|광주광역시|광주광역시|-|수치형|
|제주특별자치도|제주특별자치도|-|수치형|
|알수없음|알수없음|-|수치형|

In [848]:
df_region_accrual[['질병코드', '질병명']] = df_region_accrual['질병코드 및 질환별(1)'].apply(split_codes)

In [849]:
df_region_accrual.columns

Index(['질병코드 및 질환별(1)', '발생자 수', '서울특별시', '인천광역시', '경기도', '강원도', '충청남도',
       '충청북도', '대전광역시', '세종특별자치시', '경상남도', '경상북도', '부산광역시', '대구광역시', '울산광역시',
       '전라북도', '전라남도', '광주광역시', '제주특별자치도', '알수없음', '질병코드', '질병명'],
      dtype='object')

In [850]:
df_region_accrual = df_region_accrual[['질병코드', '질병명', '발생자 수', '서울특별시', '인천광역시', '경기도', '강원도', '충청남도',
       '충청북도', '대전광역시', '세종특별자치시', '경상남도', '경상북도', '부산광역시', '대구광역시', '울산광역시',
       '전라북도', '전라남도', '광주광역시', '제주특별자치도', '알수없음']]

In [851]:
df_region_accrual[:5]

,질병코드,질병명,발생자 수,서울특별시,인천광역시,경기도,강원도,충청남도,충청북도,대전광역시,...,경상남도,경상북도,부산광역시,대구광역시,울산광역시,전라북도,전라남도,광주광역시,제주특별자치도,알수없음
0,계,NaN,43079,8601,2446,11377,1274,1752,1291,1080,...,2431,2243,2762,2120,789,1501,1392,1076,690,16
1,D61.3,특발성 무형성빈혈,406,73,29,101,20,17,9,12,...,24,19,15,22,6,6,20,20,9,1
2,D61.9,저형성빈혈 NOS,223,47,34,56,4,9,2,1,...,21,7,15,11,4,6,2,1,1,0
3,D68.6,항인지질증후군,469,126,27,121,12,20,3,10,...,11,30,13,37,4,13,13,14,14,0
4,D86.0,폐의 사르코이드증,319,79,24,96,5,14,10,4,...,11,12,15,13,5,6,12,9,3,0


In [900]:
# CSV 파일로 내보내기
df_region_accrual.to_csv('./New_csv(데이터전처리 완료된 csv)/2021_지역별_질병분류코드별_발생자수.csv', index=False, encoding='utf-8')

#### 전국 질병분류코드별 연령군별 사망자수 data

In [914]:
df_age_death = pd.read_csv('./csv/2021_전국_질병분류코드별_연령군별_사망자_수_20240424100309.csv')

##### Data 살펴보기

|value|contents|key|opinion|
|--|--|--|--|
|질병분류코드 및 질환별(1)|희귀질환 종류|희귀질환, 극희귀질환, 기타염색체이상질환|범주형|
|질병분류코드 및 질환별(2)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병|-|범주형, 코드와 질환명 분류 필요|
|사망자수|사망자수|-|수치형|
|<1|1세 미만|-|수치형|
|1-14|1에서 14세|-|수치형|
|15-24|15에서 24세|-|수치형|
|25-44|25에서 44세|-|수치형|
|45-64|45에서 64세|-|수치형|
|65≤|65세 이상|-|수치형|

In [854]:
df_age_death.columns = df_age_death.iloc[0]

In [855]:
df_age_death = df_age_death[1:].reset_index().drop(columns=['index'])
df_age_death

,질병분류코드 및 질환별(1),질병분류코드 및 질환별(2),사망자수,<1,1-14,15-24,25-44,45-64,65≤
0,계,소계,1845,34,11,15,65,344,1376
1,희귀질환,소계,1826,30,11,14,65,337,1369
2,희귀질환,A81.0 크로이츠펠트-야콥병,29,0,0,0,0,8,21
3,희귀질환,D59.3 비정형 용혈-요독증후군,4,0,0,0,0,0,4
4,희귀질환,D59.5 발작성 야간헤모글로빈뇨,2,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...
152,극희귀질환,Q87.0 가부키 증후군,1,1,0,0,0,0,0
153,극희귀질환,Q89.7 ARC 증후군,1,1,0,0,0,0,0
154,극희귀질환,질병의 특성상 다양한 장기에 다양한 병변이 발생되는 질환으로 진단을 통해 확인된 병...,2,0,0,0,0,0,2
155,극희귀질환,Q87.5 코핀-시리스 증후군,1,1,0,0,0,0,0


In [856]:
df_age_death[['질병코드', '질병명']] = df_age_death['질병분류코드 및 질환별(2)'].apply(split_codes)

In [857]:
df_age_death = df_age_death.rename(columns={'질병분류코드 및 질환별(1)':'질환분류'})

In [858]:
df_age_death = df_age_death[['질환분류', '질병코드', '질병명', '사망자수', '<1', '1-14', '15-24', '25-44',
       '45-64', '65≤']]

In [859]:
df_age_death.columns = ['질환분류', '질병코드', '질병명', '사망자수', '1세미만', '1~14세', '15-24세', '25-44세',
       '45-64세', '65세 이상']

In [860]:
# CSV 파일로 내보내기
df_age_death.to_csv('./New_csv(데이터전처리 완료된 csv)/2021_전국_질병분류코드별_연령군별_사망자_수.csv', index=False, encoding='utf-8')

#### 지역별 질병분류코드별 사망자수 data

In [861]:
df_region_death = pd.read_csv('./csv/2021_지역별_질병분류코드별_사망자_수_20240424100217.csv')

|value|contents|key|opinion|
|--|--|--|--|
|질병분류코드 및 질환별(1)|희귀질환 종류|희귀질환, 극희귀질환, 기타염색체이상질환, 계|범주형, 계 정보 따로 빼야 함.|
|질병분류코드 및 질환별(2)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병, 소계|-|범주형, 코드와 질환명 분류 필요, 소계 정보 따로 빼야 함|
|사망자수|사망자수|-|수치형|
|서울특별시|서울특별시|-|수치형|
|인천광역시|인천광역시|-|수치형|
|경기도|경기도|-|수치형|
|강원도|강원도|-|수치형|
|충청남도|충청남도|-|수치형|
|충청북도|충청북도|-|수치형|
|대전광역시|대전광역시|-|수치형|
|세종특별자치시|세종특별자치시|-|수치형|
|부산광역시|부산광역시|-|수치형|
|대구광역시|대구광역시|-|수치형|
|울산광역시|울산광역시|-|수치형|
|전라북도|전라북도|-|수치형|
|전라남도|전라남도|-|수치형|
|광주광역시|광주광역시|-|수치형|
|제주특별자치도|제주특별자치도|-|수치형|

In [862]:
df_region_death.columns = df_region_death.iloc[0]

In [863]:
df_region_death = df_region_death.rename(columns={'질병분류코드 및 질환별(1)':'index'})
df_region_death = df_region_death.set_index('index')

In [864]:
df_region_death = df_region_death.iloc[2:].reset_index()

In [865]:
df_region_death

,index,질병분류코드 및 질환별(2),사망자 수,서울특별시,인천광역시,경기도,강원도,충청남도,충청북도,대전광역시,세종특별자치시,경상남도,경상북도,부산광역시,대구광역시,울산광역시,전라북도,전라남도,광주광역시,제주특별자치도
0,희귀질환,소계,1826,332,104,450,76,77,75,36,14,104,87,133,85,31,84,76,38,24
1,희귀질환,A81.0 크로이츠펠트-야콥병,29,6,2,11,0,0,0,1,0,1,0,3,0,0,0,3,2,0
2,희귀질환,D59.3 비정형 용혈-요독증후군,4,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
3,희귀질환,D59.5 발작성 야간헤모글로빈뇨,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0
4,희귀질환,D60.0 만성 후천성 순수적혈구무형성,4,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,극희귀질환,Q87.0 가부키 증후군,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
152,극희귀질환,Q89.7 ARC 증후군,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
153,극희귀질환,질병의 특성상 다양한 장기에 다양한 병변이 발생되는 질환으로 진단을 통해 확인된 병...,2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
154,극희귀질환,Q87.5 코핀-시리스 증후군,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 진료비용, 보험청구비용 data 

#### 희귀질환 data

##### 질병분류코드 + 요양급여비용 항목별 진료내역 data

In [866]:
df_medical_expense = pd.read_csv('./csv/2021_질병분류코드별_진료내역_항목별_요양급여비용_현황_등록_후_12개월__20240424095654.csv')

|value|contents|key|opinion|
|--|--|--|--|
|질병분류코드 및 질환별(1)|희귀질환 종류|희귀질환, 극희귀질환, 기타염색체이상질환|범주형|
|질병분류코드 및 질환별(2)|질환코드/질환명 분류 ex)A81.0 크로이츠펠트-야콥병|-|범주형, 코드와 질환명 분류 필요|
|계|아래 항목들의 총액|-|수치형|
|진찰료|진찰료|-|수치형|
|입원료|입원료|-|수치형|
|투약료|투약료|-|수치형|
|주사료|주사료|-|수치형|
|마취료|마취료|-|수치형|
|이학요법료|물리치료료-비급여|-|수치형|
|정신요법료|정신치료- 비급여|-|수치형|
|처치 및 수술료|처치 및 수술료 - 비급여|-|수치형|
|검사료|검사료 - 비급여|-|수치형|
|영상 및 방사선 치료료|영상 및 방사선 치료료 - 비급여|-|수치형|
|기타|기타|-|수치형|

In [867]:
df_medical_expense.columns = df_medical_expense.iloc[0]

In [868]:
df_medical_expense = df_medical_expense.iloc[1:]

In [869]:
df_medical_expense = df_medical_expense.reset_index()
df_medical_expense.drop(columns='index', inplace=True)

In [870]:
df_medical_expense.columns = df_medical_expense.columns.str.replace('질병분류코드 및 질환별(1)', '질환분류')
df_medical_expense.columns = df_medical_expense.columns.str.replace('질병분류코드 및 질환별(2)', '질환종류')

In [871]:
df_medical_expense['질환코드']= df_medical_expense['질환종류'].apply(split_codes)[0]
df_medical_expense['질환명'] = df_medical_expense['질환종류'].apply(split_codes)[1]

In [872]:
df_medical_expense.drop(columns='질환종류', inplace=True)

In [873]:
df_medical_expense = df_medical_expense[['질환분류', '질환코드', '질환명', '계', '진찰료', '입원료', '투약료', '주사료', '마취료', '이학요법료', '정신요법료',
       '처치 및 수술료', '검사료', '영상 및 방사선 치료료', '기타']]

In [874]:
df_medical_expense.to_csv('./New_csv(데이터전처리 완료된 csv)/2021_질병분류코드별_진료내역_항목별_요양급여비용_현황.csv', index=False, encoding='utf-8')

##### 질병분류코드별 건강보험 청구 및 요양급여비용 내역 data

###### 희귀질환 진료비 통계

In [904]:
df_health_insurance_2021 = pd.read_csv('./csv/건강보험심사평가원_희귀질환 진료비 통계 2021.csv', encoding='CP949')

|value|contents|key|opinion|
|--|--|--|--|
|진료년도|연도 정보|2021|2021년도 정보밖에 없으니 삭제|
|희귀질환 상병코드|질환별 코드|A00 - Q99|opinion|
|환자수|환자수|-|수치형|
|명세서 청구건수|명세서 청구건수|-|수치형|
|입내원일수|입내원일수|-|수치형|
|보험자부담금|건강보험료|-|수치형|
|요양급여비용총액|요양급여비용총액|-|수치형|

In [905]:
def insert_comma(x):
    if len(x)==3 :
        result = x+'.0'
    elif len(x)==4 :
        result = x[:3]+'.'+x[-1]
    elif len(x)==5 :
        result = x[:3]+'.'+x[-2:]
    return result

In [906]:
df_health_insurance_2021['희귀질환 상병코드'] = df_health_insurance_2021['희귀질환 상병코드'].apply(insert_comma)

In [909]:
df_health_insurance_2021.drop(columns='진료년도').to_csv('./New_csv(데이터전처리 완료된 csv)/심평원_희귀질환_건강보험_진료_통계_2021.csv', index=False, encoding='utf-8')

###### 건강보험 진료 통계

In [879]:
df_health_insurance_2022 = pd.read_csv('./csv/건강보험심사평가원_희귀질환 건강보험 진료 통계_20221231.csv', encoding='CP949')

|value|contents|key|opinion|
|--|--|--|--|
|진료년도|연도 정보|2021|2021년도 정보밖에 없으니 삭제|
|희귀질환 상병코드|질환별 코드|A00 - Q99|opinion|
|환자수|환자수|-|수치형|
|명세서 청구건수|명세서 청구건수|-|수치형|
|입내원일수|입내원일수|-|수치형|
|보험자부담금|건강보험료|-|수치형|
|요양급여비용총액|요양급여비용총액|-|수치형|

In [880]:
df_health_insurance_2022['희귀질환 상병코드'] = df_health_insurance_2022['희귀질환 상병코드'].apply(insert_comma)

In [881]:
df_health_insurance_2022.drop(columns='진료년도').to_csv('./New_csv(데이터전처리 완료된 csv)/심평원_희귀질환_건강보험_진료_통계_2022.csv', index=False, encoding='utf-8')

#### 중증질환 data

##### 4대 중증질환(암, 뇌혈관, 심장, 휘귀/중증난치질환) 약품비 data

In [882]:
df_serious_disease_medicine = pd.read_csv('./csv/2022_4대_중증질환_약품비_건강보험__20240424094151.csv')


|value|contents|key|opinion|
|--|--|--|--|
|중증질환 구분(1)|중증질환 구분|'계', '암질환', '뇌혈관질환', '심장질환', '희귀·중증난치질환'|계 제외하기|
|계 - 청구건수(천건)|전체 청구건수|A00 - Q99|opinion|
|계 - 청구금액(백만원)|전체 청구금액|-|수치형|
|입원 - 청구건수(천건)	|입원 청구건수|-|수치형|
|입원 - 청구금액(백만원)	|입원 청구금액|-|수치형|
|외래 - 청구건수(천건)|외래 청구건수|-|수치형|
|외래 - 청구금액(백만원)|외래 청구금액|-|수치형|

In [883]:
df_serious_disease_medicine.columns = df_serious_disease_medicine.iloc[0]+df_serious_disease_medicine.iloc[1]

In [884]:
df_serious_disease_medicine

,중증질환 구분(1)중증질환 구분(1),계청구건수 (천건),계청구금액 (백만원),입원청구건수 (천건),입원청구금액 (백만원),외래청구건수 (천건),외래청구금액 (백만원)
0,중증질환 구분(1),계,계,입원,입원,외래,외래
1,중증질환 구분(1),청구건수 (천건),청구금액 (백만원),청구건수 (천건),청구금액 (백만원),청구건수 (천건),청구금액 (백만원)
2,계,25069.0,5849552,2376.0,1440835,22692.0,4408717
3,암질환,9827.0,3117039,1595.0,1068558,8232.0,2048481
4,뇌혈관질환,78.0,65588,77.0,65511,0.6,77
5,심장질환,110.0,55247,107.0,54856,3.0,390
6,희귀·중증난치질환,15054.0,2611679,597.0,251910,14457.0,2359769


In [885]:
df_serious_disease_medicine.columns.str.replace('계','총')
df_serious_disease_medicine.columns = df_serious_disease_medicine.columns.str.replace('중증질환 구분(1)중증질환 구분(1)', 'index')
df_serious_disease_medicine.set_index('index', inplace=True)

In [886]:
df_serious_disease_medicine = df_serious_disease_medicine.iloc[2:]

In [887]:
df_serious_disease_medicine.to_csv('./New_csv(데이터전처리 완료된 csv)/2022_4대_중증질환_약품비_건강보험.csv', index=True, encoding='utf-8')

##### 중증질환(암, 뇌혈관, 심장, 휘귀/중증난치질환, 화상, 외상, 결핵) 진료비용 data

In [888]:
df_gender_age_serious_disease_medical_expense = pd.read_csv('./csv/2022_중증질환_산정특례적용_연령별_성별_진료현황_20240424093755.csv')

###### Data 살펴보기

전체적으로 전처리 필수
|value|contents|key|opinion|
|--|--|--|--|
|연령별(1)|중증질환 구분|-|계 제외하기|
|성별(1)|전체 청구건수|-|opinion|
|계 - 진료실인원수 (명)|총 진료실인원수(명)|-|수치형|
|계 - 입내원일수 (일)|총 입내원일수 (일)|-|수치형|
|계 -	요양급여일수 (일)|총 요양급여일수 (일)|-|수치형|
|계 - 진료비 (천원)	|총 진료비 (천원)|-|수치형|
|계 - 급여비 (천원)|총 급여비 (천원)|-|수치형|
|암질환 - 진료실인원수 (명)|암질환의 진료실인원수 (명)|-|수치형|
|암질환 - 입내원일수 (일)|암질환의 입내원일수 (일|-|수치형|
|암질환 - 요양급여일수 (일)|암질환의 요양급여일수 (일)|-|수치형|
|암질환 -  진료비 (천원)|암질환의 진료비 (천원)|-|수치형|
|암질환 -  급여비 (천원)|암질환의 급여비 (천원)|-|수치형|
|뇌혈관 - 진료실인원수 (명)|뇌혈관의 진료실인원수 (명)|-|수치형|
|뇌혈관 - 입내원일수 (일)|뇌혈관의 입내원일수 (일|-|수치형|
|뇌혈관 - 요양급여일수 (일)|뇌혈관의 요양급여일수 (일)|-|수치형|
|뇌혈관 -  진료비 (천원)|뇌혈관의 진료비 (천원)|-|수치형|
|뇌혈관 -  급여비 (천원)|뇌혈관의 급여비 (천원)|-|수치형|
|희귀난치성질환 - 진료실인원수 (명)|희귀난치성질환의 진료실인원수 (명)|-|수치형|
|희귀난치성질환 - 입내원일수 (일)|희귀난치성질환의 입내원일수 (일|-|수치형|
|희귀난치성질환 - 요양급여일수 (일)|희귀난치성질환의 요양급여일수 (일)|-|수치형|
|희귀난치성질환 -  진료비 (천원)|희귀난치성질환의 진료비 (천원)|-|수치형|
|희귀난치성질환 -  급여비 (천원)|희귀난치성질환의 급여비 (천원)|-|수치형|
|중증화상 - 진료실인원수 (명)|중증화상의 진료실인원수 (명)|-|수치형|
|중증화상 - 입내원일수 (일)|중증화상의 입내원일수 (일|-|수치형|
|중증화상 - 요양급여일수 (일)|중증화상의 요양급여일수 (일)|-|수치형|
|중증화상 -  진료비 (천원)|중증화상의 진료비 (천원)|-|수치형|
|중증화상 -  급여비 (천원)|중증화상의 급여비 (천원)|-|수치형|
|결핵 - 진료실인원수 (명)|결핵의 진료실인원수 (명)|-|수치형|
|결핵 - 입내원일수 (일)|결핵의 입내원일수 (일|-|수치형|
|결핵 - 요양급여일수 (일)|결핵의 요양급여일수 (일)|-|수치형|
|결핵 -  진료비 (천원)|결핵의 진료비 (천원)|-|수치형|
|결핵 -  급여비 (천원)|결핵의 급여비 (천원)|-|수치형|
|잠복결핵 - 진료실인원수 (명)|잠복결핵의 진료실인원수 (명)|-|수치형|
|잠복결핵 - 입내원일수 (일)|잠복결핵의 입내원일수 (일|-|수치형|
|잠복결핵 - 요양급여일수 (일)|잠복결핵의 요양급여일수 (일)|-|수치형|
|잠복결핵 -  진료비 (천원)|잠복결핵의 진료비 (천원)|-|수치형|
|잠복결핵 -  급여비 (천원)|잠복결핵의 급여비 (천원)|-|수치형|


In [889]:
df_gender_age_serious_disease_medical_expense = df_gender_age_serious_disease_medical_expense.T
df_gender_age_serious_disease_medical_expense.set_index(0, inplace=True)
df_gender_age_serious_disease_medical_expense = df_gender_age_serious_disease_medical_expense.rename_axis('', axis=1)
df_gender_age_serious_disease_medical_expense.rename({'연령별(1)':'연령별'})

,1,2,3,4,5,6,7,8,9,10,...,52,53,54,55,56,57,58,59,60,61
0,,,,,,,,,,,,,,,,,,,,,
연령별,연령별(1),계,계,계,0세,0세,0세,1 - 4세,1 - 4세,1 - 4세,...,70 - 74세,75 - 79세,75 - 79세,75 - 79세,80 - 84세,80 - 84세,80 - 84세,85세 이상,85세 이상,85세 이상
성별(1),성별(1),계,남자,여자,계,남자,여자,계,남자,여자,...,여자,계,남자,여자,계,남자,여자,계,남자,여자
계,진료실인원수 (명),2569669,1213016,1356653,2490,1431,1059,10569,5800,4769,...,117167,233233,128932,104301,205823,104267,101556,142675,59330,83345
계,입내원일수 (일),67044120,32266545,34777575,19856,10387,9469,214455,115678,98777,...,3366065,7192032,3905416,3286616,7082854,3399994,3682860,5430711,2070423,3360288
계,요양급여일수 (일),617580504,278510470,339070034,95527,50772,44755,1234406,651838,582568,...,31360021,57873766,31038806,26834960,49743548,25402178,24341370,31686438,13504612,18181826
계,진료비 (천원),20955584856,11274973918,9680610939,8051096,3959615,4091481,44073489,25020111,19053378,...,996661993,2172329097,1264084214,908244883,1867023669,985578735,881444933,1165498719,516736720,648761999
계,급여비 (천원),19026420452,10269419580,8757000874,7457185,3657134,3800051,40073728,22676661,17397067,...,907017808,1974497146,1151486644,823010502,1691286896,896321142,794965754,1046881155,466603730,580277425
암질환,진료실인원수 (명),1370510,633417,737093,40,20,20,882,456,426,...,61093,130347,81340,49007,103535,62877,40658,60497,33506,26991
암질환,입내원일수 (일),33663687,15917584,17746103,1344,804,540,38854,19006,19848,...,1575564,3475218,2141007,1334211,3066528,1777921,1288607,2206927,1066104,1140823


In [890]:
df_gender_age_serious_disease_medical_expense.columns = df_gender_age_serious_disease_medical_expense.iloc[0]

In [891]:
df_gender_age_serious_disease_medical_expense.index = df_gender_age_serious_disease_medical_expense.index+ df_gender_age_serious_disease_medical_expense['연령별(1)']

In [892]:
df_gender_age_serious_disease_medical_expense=df_gender_age_serious_disease_medical_expense[1:]

In [893]:
df_gender_age_serious_disease_medical_expense.index = df_gender_age_serious_disease_medical_expense.index.str.replace('(1)', '')
df_gender_age_serious_disease_medical_expense.index = df_gender_age_serious_disease_medical_expense.index.str.replace('계', '총')
df_gender_age_serious_disease_medical_expense.rename_axis('index', inplace=True, axis='columns')

In [894]:
df_gender_age_serious_disease_medical_expense.columns = df_gender_age_serious_disease_medical_expense.columns.str.replace('(1)', '')

In [895]:
df_gender_age_serious_disease_medical_expense_new = df_gender_age_serious_disease_medical_expense.T.query('성별성별!="계"')

In [896]:
df_gender_age_serious_disease_medical_expense_new = df_gender_age_serious_disease_medical_expense_new.T.rename({'성별성별':''})

In [897]:
df_gender_age_serious_disease_medical_expense_new.drop(columns='연령별')

index,계,계,0세,0세,1 - 4세,1 - 4세,5 - 9세,5 - 9세,10 - 14세,10 - 14세,...,65 - 69세,65 - 69세,70 - 74세,70 - 74세,75 - 79세,75 - 79세,80 - 84세,80 - 84세,85세 이상,85세 이상
,남자,여자,남자,여자,남자,여자,남자,여자,남자,여자,...,남자,여자,남자,여자,남자,여자,남자,여자,남자,여자
총진료실인원수 (명),1213016,1356653,1431,1059,5800,4769,7859,6606,8081,6772,...,163000,141911,149917,117167,128932,104301,104267,101556,59330,83345
총입내원일수 (일),32266545,34777575,10387,9469,115678,98777,158904,126585,137275,112926,...,4660729,3786218,4423727,3366065,3905416,3286616,3399994,3682860,2070423,3360288
총요양급여일수 (일),278510470,339070034,50772,44755,651838,582568,1251906,1032220,1502564,1215746,...,38452237,38743400,35904715,31360021,31038806,26834960,25402178,24341370,13504612,18181826
총진료비 (천원),11274973918,9680610939,3959615,4091481,25020111,19053378,38212406,21917021,52201115,26817487,...,1663988849,1137320401,1516391375,996661993,1264084214,908244883,985578735,881444933,516736720,648761999
총급여비 (천원),10269419580,8757000874,3657134,3800051,22676661,17397067,34652437,19945420,47438356,24338470,...,1521311788,1036253596,1385238212,907017808,1151486644,823010502,896321142,794965754,466603730,580277425
암질환진료실인원수 (명),633417,737093,20,20,456,426,1032,831,1117,1007,...,99792,79408,94412,61093,81340,49007,62877,40658,33506,26991
암질환입내원일수 (일),15917584,17746103,804,540,19006,19848,32923,26178,36007,29735,...,2605554,2012406,2459203,1575564,2141007,1334211,1777921,1288607,1066104,1140823
암질환요양급여일수 (일),127206540,161524882,1454,1758,60669,60470,151877,117110,179058,156615,...,20295003,18111572,20051294,13414118,17978636,10365007,14483962,7827682,7567106,4597540
암질환진료비 (천원),5656724418,5472834004,364848,105520,6050999,6709639,9211070,6872562,11710432,8696486,...,962290372,668568683,889761026,530503049,730027467,427257862,532254632,350230886,255429284,225296542


In [898]:
# CSV 파일로 내보내기
df_gender_age_serious_disease_medical_expense_new.to_csv('./New_csv(데이터전처리 완료된 csv)/2022_중증질환_산정특례적용_연령별_성별_진료현황.csv', index=True, encoding='utf-8')

### 인구통계 data

#### 대한민국 도시지역 인구통계

In [920]:
df_population = pd.read_csv('./csv/2021_2022_대한민국_도시지역_인구현황_시군구__20240502095346.csv', header=2)
df_population[:5]

,소재지(시군구)별(1),소재지(시군구)별(2),소계,도시인구(B),비도시인구(C=A-B),도시인구(D),농촌인구(E=A-D),용도지역 인구기준(B/A*100),행정구역 인구기준(D/A*100),소계.1,도시인구(B).1,비도시인구(C=A-B).1,도시인구(D).1,농촌인구(E=A-D).1,용도지역 인구기준(B/A*100).1,행정구역 인구기준(D/A*100).1
0,전국,소계,51638809,47402647,4236162,47180828,4457981,91.8,91.37,51439038,47294436,4144602,47038612,4400426,91.94,91.45
1,서울특별시,소계,9509458,9509458,0,9509458,0,100.0,100.00,9428372,9428372,-,9428372,-,100.00,100.00
2,서울특별시,종로구,144683,144683,0,144683,0,100.0,100.00,141379,141379,-,141379,-,100.00,100.00
3,서울특별시,중구,122499,122499,0,122499,0,100.0,100.00,120437,120437,-,120437,-,100.00,100.00
4,서울특별시,용산구,222953,222953,0,222953,0,100.0,100.00,218650,218650,-,218650,-,100.00,100.00


In [937]:
# 2021년 대한민국 인구통계
df_population_2021 = df_population.iloc[:, 0:9]
df_population_2021 = df_population_2021.rename(columns = {'소재지(시군구)별(1)': '행정구역', '소재지(시군구)별(2)': '지역구', '도시인구(B)': '용도지역기준(도시인구)', '비도시인구(C=A-B)': '용도지역기준(비도시인구)', '도시인구(D)': '행정구역기준(도시인구)', '농촌인구(E=A-D)': '행정구역기준(농촌인구)', '용도지역 인구기준(B/A*100)': '용도지역 인구기준(도시지역 인구비율(%))', '행정구역 인구기준(D/A*100)': '행정구역 인구기준(도시지역 인구비율(%))'})
df_population_2021

,행정구역,지역구,소계,용도지역기준(도시인구),용도지역기준(비도시인구),행정구역기준(도시인구),행정구역기준(농촌인구),용도지역 인구기준(도시지역 인구비율(%)),행정구역 인구기준(도시지역 인구비율(%))
0,전국,소계,51638809,47402647,4236162,47180828,4457981,91.80,91.37
1,서울특별시,소계,9509458,9509458,0,9509458,0,100.00,100.00
2,서울특별시,종로구,144683,144683,0,144683,0,100.00,100.00
3,서울특별시,중구,122499,122499,0,122499,0,100.00,100.00
4,서울특별시,용산구,222953,222953,0,222953,0,100.00,100.00
...,...,...,...,...,...,...,...,...,...
242,경상남도,거창군,61073,40677,20396,40541,20532,66.60,66.38
243,경상남도,합천군,42935,24728,18207,11108,31827,57.59,25.87
244,제주특별자치도,소계,676759,612755,64004,639565,37194,90.54,94.50
245,제주특별자치도,제주시,493096,453648,39448,480523,12573,92.00,97.45


In [938]:
# 2022년 대한민국 인구통계만 추출
df_population_2022 = df_population.drop(df_population.columns[2:9], axis=1)
df_population_2022 = df_population_2022.rename(columns = {'소재지(시군구)별(1)': '행정구역', '소재지(시군구)별(2)': '지역구', '소계.1': '소계', '도시인구(B).1': '용도지역기준(도시인구)', '비도시인구(C=A-B).1': '용도지역기준(비도시인구)', '도시인구(D).1': '행정구역기준(도시인구)', '농촌인구(E=A-D).1': '행정구역기준(농촌인구)', '용도지역 인구기준(B/A*100).1': '용도지역 인구기준(도시지역 인구비율(%))', '행정구역 인구기준(D/A*100).1': '행정구역 인구기준(도시지역 인구비율(%))'})
df_population_2022

,행정구역,지역구,소계,용도지역기준(도시인구),용도지역기준(비도시인구),행정구역기준(도시인구),행정구역기준(농촌인구),용도지역 인구기준(도시지역 인구비율(%)),행정구역 인구기준(도시지역 인구비율(%))
0,전국,소계,51439038,47294436,4144602,47038612,4400426,91.94,91.45
1,서울특별시,소계,9428372,9428372,-,9428372,-,100.00,100.00
2,서울특별시,종로구,141379,141379,-,141379,-,100.00,100.00
3,서울특별시,중구,120437,120437,-,120437,-,100.00,100.00
4,서울특별시,용산구,218650,218650,-,218650,-,100.00,100.00
...,...,...,...,...,...,...,...,...,...
242,경상남도,거창군,60387,40220,20167,39933,20454,66.60,66.13
243,경상남도,합천군,42142,24270,17872,11005,31137,57.59,26.11
244,제주특별자치도,소계,678159,613564,64595,640477,37682,90.47,94.44
245,제주특별자치도,제주시,493389,453794,39595,480647,12742,91.97,97.42


In [943]:
df_population_2022.columns

Index(['행정구역', '지역구', '소계', '용도지역기준(도시인구)', '용도지역기준(비도시인구)', '행정구역기준(도시인구)',
       '행정구역기준(농촌인구)', '용도지역 인구기준(도시지역 인구비율(%))', '행정구역 인구기준(도시지역 인구비율(%))'],
      dtype='object')

In [944]:
df_population_2022['지역구'].value_counts()

지역구
소계     18
동구      6
중구      6
서구      5
북구      4
       ..
안성시     1
포천시     1
의왕시     1
하남시     1
파주시     1
Name: count, Length: 208, dtype: int64

##### Data 살펴보기

|value|contents|key|opinion|
|--|--|--|--|
|행정구역|대한민국 행정구역|경기도, 서울특별시, 경상북도, 전라남도, 경상남도, 강원도, 부산광역시, 충청남도, 전라북도 등|범주형|
|지역구|대한민국 각 행적구역 별 지역구|동구, 중구, 서구, 북구, 안성시, 포천시, 하남시 등|범주형|
|용도지역기준(도시인구)|용도지역기준으로 분류된 도시인구|-|수치형|
|용도지역기준(비도시인구)|용도지역기준으로 분류된 비도시인구|-|수치형|
|행정구역기준(도시인구)|행정구역으로 분류된 도시인구|-|수치형|
|행정구역기준(농촌인구)|행정구역으로 분류된 농촌인구|-|수치형|
|용도지역 인구기준(도시지역 인구비율(%))|용도지역기준 도시인구 비율|-|수치형|
|행정구역 인구기준(도시지역 인구비율(%))|행정구역기준 도시인구 비율|-|수치형|

In [945]:
# CSV 파일로 내보내기
df_population_2021.to_csv('./New_csv(데이터전처리 완료된 csv)/2021_대한민국_인구통계.csv', index=False, encoding='utf-8')
df_population_2022.to_csv('./New_csv(데이터전처리 완료된 csv)/2022_대한민국_인구통계.csv', index=False, encoding='utf-8')

#### 대한민국 성별/연령별 인구 통계

In [967]:
df_gender_age_population = pd.read_csv('./csv/2021_2022_대한민국_성_및_연령별_추계인구_1세별__5세별____전국_20240502095104.csv')
df_gender_age_population[:5]

,가정별,성별,연령별,2021,2022
0,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제순이동-중위),전체,계,51769539,51672569
1,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제순이동-중위),전체,0 - 4세,1618830,1494041
2,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제순이동-중위),전체,5 - 9세,2231797,2142084
3,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제순이동-중위),전체,10 - 14세,2297714,2311839
4,중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제순이동-중위),전체,15 - 19세,2345886,2287873


In [968]:
df_gender_age_population['가정별'].value_counts()

가정별
중위 추계(기본 추계: 출산율-중위 / 기대수명-중위 / 국제순이동-중위)    69
Name: count, dtype: int64

##### Data 살펴보기

| value | contents | key | opinion |
| ----- | -------- | --- | ------- |
| 가정별 | ? | 중위 추계(기본 추계: 출산율-중위/기대수명-중위/국제순이동-중위) | 필요없어서 제외 |
| 성별 | 성별 | 전체, 남자, 여자 | 범주형 |
| 연령별 | 연령 | 계, 0-4세, 5-9세, 10-14세, 15-19세, 20-24세 등 | 범주형 |
| 2021 | 2021년 인구통계 | - | 수치형 |
| 2022 | 2022년 인구통계 | - | 수치형 |

In [969]:
# '가정별' column 삭제
df_gender_age_population = df_gender_age_population.drop(df_gender_age_population.columns[0], axis=1)
df_gender_age_population

,성별,연령별,2021,2022
0,전체,계,51769539,51672569
1,전체,0 - 4세,1618830,1494041
2,전체,5 - 9세,2231797,2142084
3,전체,10 - 14세,2297714,2311839
4,전체,15 - 19세,2345886,2287873
...,...,...,...,...
64,여자,80 - 84세,719722,762492
65,여자,85 - 89세,421203,445118
66,여자,90 - 94세,158017,168603
67,여자,95 - 99세,35223,36739


In [970]:
# 2021년도만 추출
df_gender_age_population_2021 = df_gender_age_population.drop(df_gender_age_population.columns[3], axis=1)
df_gender_age_population_2021

,성별,연령별,2021
0,전체,계,51769539
1,전체,0 - 4세,1618830
2,전체,5 - 9세,2231797
3,전체,10 - 14세,2297714
4,전체,15 - 19세,2345886
...,...,...,...
64,여자,80 - 84세,719722
65,여자,85 - 89세,421203
66,여자,90 - 94세,158017
67,여자,95 - 99세,35223


In [971]:
# 2022년도만 추출
df_gender_age_population_2022 = df_gender_age_population.drop(df_gender_age_population.columns[2], axis=1)
df_gender_age_population_2022

,성별,연령별,2022
0,전체,계,51672569
1,전체,0 - 4세,1494041
2,전체,5 - 9세,2142084
3,전체,10 - 14세,2311839
4,전체,15 - 19세,2287873
...,...,...,...
64,여자,80 - 84세,762492
65,여자,85 - 89세,445118
66,여자,90 - 94세,168603
67,여자,95 - 99세,36739


In [972]:
# CSV 파일로 내보내기
df_gender_age_population_2021.to_csv('./New_csv(데이터전처리 완료된 csv)/2021_대한민국_성_및_연령별_인구통계.csv', index=False, encoding='utf-8')
df_gender_age_population_2022.to_csv('./New_csv(데이터전처리 완료된 csv)/2022_대한민국_성_및_연령별_인구통계.csv', index=False, encoding='utf-8')